In [1]:
import plotly.graph_objects as go

In [301]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 0,
      thickness = 10,
      line = dict(color = "black", width = 0.5),
      label = ['1' + k for k in d['descritpion']] + ['2' + k for k in d['descritpion']] + ['3' + k for k in d['descritpion']] + ['4' + k for k in d['descritpion']],
      color = d['color'] + d['color'] + d['color'] + d['color']
    ),
    link = dict(
        source = from_complete_links + to_complete_links + to_to_complete_links, # indices correspond to labels, eg A1, A2, A1, B1, ...
        target = to_complete_links + to_to_complete_links + to_to_to_complete_links,
        value = [1] * len(from_complete_links) * 3,
        color = ['white'] * len(from_complete_links) * 3
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)

fig.write_html("flow.html")

In [ ]:
%%html
<iframe src="flow.html" width="800" height="600" title="ridgeline chart with plotly" style="border:none"></iframe>

In [168]:
def find_consecutive_from_idx(lst, idx):
    if idx + 1 == len(lst):
        return idx
    elif idx + 1 > len(lst):
        return None
    
    for i in range(idx, len(lst)):
        if not i + 1 in range(len(lst)):
            return i
        if lst[i] != lst[i + 1] - 1:
            return i

In [283]:
def create_split(list_of_mutations, n):
    d = {
        'label' : [],
        'from' : [],
        'to' : [],
        'descritpion' : [],
        'color' : []
    }
    k = 0
    L = 0
    index = 0
    while index in range(len(list_of_mutations)):
        if index == 0 and list_of_mutations[0] != 0:
            d['label'].append(k)
            d['from'].append(L)
            d['to'].append(list_of_mutations[0]-1)
            d['descritpion'].append("From {}, to {}".format(L, list_of_mutations[0]-1))
            d['color'].append('white')
            k = k + 1
        L = list_of_mutations[index]
        
        H_idx = find_consecutive_from_idx(list_of_mutations, index)
        if H_idx == None:
            break    
        H = list_of_mutations[H_idx]
        index = H_idx + 1
        d['label'].append(k)
        d['from'].append(L)
        d['to'].append(H)
        d['descritpion'].append("From {}, to {}".format(L, H))
        d['color'].append('yellow')
        k = k + 1
        if index < len(list_of_mutations):
            d['label'].append(k)
            d['from'].append(H+1)
            d['to'].append(list_of_mutations[index]-1)
            d['descritpion'].append("From {}, to {}".format(H+1, list_of_mutations[index]-1))
            d['color'].append('white')
            k = k + 1
        elif (index == len(list_of_mutations)) and (list_of_mutations[index - 1] != n):
            d['label'].append(k)
            d['from'].append(H+1)
            d['to'].append(n)
            d['descritpion'].append("From {}, to {}".format(H+1, n))
            d['color'].append('white')
            k = k + 1
    return d

In [288]:
lst = [3,4,6,7,8,10,18,95,96,97,98,99,100] + list(range(15000, 15100))
d = create_split(lst, 30000)

In [300]:
from_complete_links = []
for idx, (_from, _to) in enumerate(zip(d['from'], d['to'])):
    _len = _to - _from + 1
    from_complete_links.extend([idx] * _len)
to_complete_links = [k + len(d['label']) for k in from_complete_links]
to_to_complete_links = [k + len(d['label']) for k in to_complete_links]
to_to_to_complete_links = [k + len(d['label']) for k in to_to_complete_links]